ORIGINAL

In [ ]:
import random
import time
import psutil
from math import gcd, isqrt

class RSA:

    def __init__(self):
        pass

    def find_random_prime(self, exclude=None):
        min_val = 2
        max_val = 999
        while True:
            candidate = random.randint(min_val, max_val)
            if self.is_prime(candidate) and candidate != exclude:
                return candidate

    def is_prime(self, num):
        if num <= 1:
            return False
        if num == 2:
            return True
        if num % 2 == 0:
            return False
        for i in range(3, isqrt(num) + 1, 2):
            if num % i == 0:
                return False
        return True

    def compute_n(self, p, q):
        return p * q

    def euler_totient(self, p, q):
        return (p - 1) * (q - 1)

    def find_e(self, z):
        for i in range(2, z):
            if gcd(i, z) == 1:
                return i

    def find_d(self, e, z):
        d = 1
        while (d * e) % z != 1:
            d += 1
        return d

    def encrypt(self, message, e, n):
        ciphertext = []
        for char in message:
            encrypted_char = pow(ord(char), e, n)
            ciphertext.append(str(encrypted_char))
        return ' '.join(ciphertext)

    def decrypt(self, ciphertext, d, n):
        message = ""
        encrypted_chars = ciphertext.split()
        for encrypted_char in encrypted_chars:
            decrypted_char = chr(pow(int(encrypted_char), d, n))
            message += decrypted_char
        return message

    def measure_encryption_speed(self, message, e, n):
        start_time = time.time()
        ciphertext = self.encrypt(message, e, n)
        end_time = time.time()
        return end_time - start_time, len(ciphertext)

    def measure_decryption_speed(self, ciphertext, d, n):
        start_time = time.time()
        message = self.decrypt(ciphertext, d, n)
        end_time = time.time()
        return end_time - start_time, len(message)

    @staticmethod
    def measure_memory_usage():
        process = psutil.Process()
        memory_usage = process.memory_info().rss
        return memory_usage

# Example usage
rsa = RSA()
p = rsa.find_random_prime()  # Choose a prime number
q = rsa.find_random_prime(p)  # Choose another prime number
n = rsa.compute_n(p, q)
z = rsa.euler_totient(p, q)
e = rsa.find_e(z)
d = rsa.find_d(e, z)
message = "Hello"

ciphertext = rsa.encrypt(message, e, n)
print("Ciphertext:", ciphertext)
decrypted_message = rsa.decrypt(ciphertext, d, n)
print("Decrypted Message:", decrypted_message)

# Measure encryption and decryption speeds
encryption_time, encrypted_length = rsa.measure_encryption_speed(message, e, n)
decryption_time, decrypted_length = rsa.measure_decryption_speed(ciphertext, d, n)
memory_usage = RSA.measure_memory_usage()

print(f"Encryption Time: {encryption_time} seconds")
print(f"Decryption Time: {decryption_time} seconds")
print(f"Memory Usage: {memory_usage} bytes")


Ciphertext: 3562 13143 78979 78979 70785
Decrypted Message: Hello
Encryption Time: 1.1920928955078125e-05 seconds
Decryption Time: 1.7642974853515625e-05 seconds
Memory Usage: 99983360 bytes


OAEP


In [ ]:
import random
import time
import psutil
from math import gcd
from hashlib import sha256

# Key Generation Functions
def generate_prime_candidate(length):
    p = random.getrandbits(length)
    p |= (1 << length - 1) | 1
    return p

def is_prime(n, k=128):
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, r, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_prime_number(length=1024):
    p = generate_prime_candidate(length)
    while not is_prime(p, 128):
        p = generate_prime_candidate(length)
    return p

def generate_keypair(bits):
    p = generate_prime_number(bits // 2)
    q = generate_prime_number(bits // 2)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(2, phi)
        g = gcd(e, phi)
    d = pow(e, -1, phi)
    return ((e, n), (d, n))

public_key, private_key = generate_keypair(1024)
print("Public Key:", public_key)
print("Private Key:", private_key)

# OAEP Encoding Functions
def mgf1(seed, mask_len, hash_function=sha256):
    h_len = hash_function().digest_size
    if mask_len > 2**32 * h_len:
        raise ValueError("mask too long")
    T = b"".join([hash_function(seed + i.to_bytes(4, byteorder="big")).digest()
                  for i in range((mask_len + h_len - 1) // h_len)])
    return T[:mask_len]

def oaep_encode(message, n_bits, label=b"", hash_function=sha256):
    k = (n_bits + 7) // 8
    m_len = len(message)
    h_len = hash_function().digest_size
    ps_len = k - m_len - 2 * h_len - 2
    if ps_len < 0:
        raise ValueError("Message too long.")
    l_hash = hash_function(label).digest()
    ps = b'\x00' * ps_len
    db = l_hash + ps + b'\x01' + message
    seed = random.randbytes(h_len)
    db_mask = mgf1(seed, k - h_len - 1, hash_function)
    masked_db = bytes(x ^ y for x, y in zip(db, db_mask))
    seed_mask = mgf1(masked_db, h_len, hash_function)
    masked_seed = bytes(x ^ y for x, y in zip(seed, seed_mask))
    return b'\x00' + masked_seed + masked_db

def oaep_decode(encoded_message, n_bits, label=b"", hash_function=sha256):
    k = (n_bits + 7) // 8
    h_len = hash_function().digest_size
    if len(encoded_message) != k or k < 2 * h_len + 2:
        raise ValueError("Decryption error.")
    l_hash = hash_function(label).digest()
    y, masked_seed, masked_db = encoded_message[0], encoded_message[1:h_len + 1], encoded_message[h_len + 1:]
    seed_mask = mgf1(masked_db, h_len, hash_function)
    seed = bytes(x ^ y for x, y in zip(masked_seed, seed_mask))
    db_mask = mgf1(seed, k - h_len - 1, hash_function)
    db = bytes(x ^ y for x, y in zip(masked_db, db_mask))
    l_hash_prime, rest = db[:h_len], db[h_len:]
    if l_hash != l_hash_prime:
        raise ValueError("Decryption error.")
    ps_end = rest.index(b'\x01')
    return rest[ps_end + 1:]

message = b"Hello, RSA with OAEP!"
print("Original Message:", message)

# OAEP Encoding
encoded_message = oaep_encode(message, 1024)
print("OAEP Encoded Message:", encoded_message)

# RSA Encryption
def rsa_encrypt(message, public_key):
    e, n = public_key
    m = int.from_bytes(message, byteorder='big')
    c = pow(m, e, n)
    return c.to_bytes((n.bit_length() + 7) // 8, byteorder='big')

encrypted_message = rsa_encrypt(encoded_message, public_key)
print("Encrypted Message:", encrypted_message)

# RSA Decryption
def rsa_decrypt(ciphertext, private_key):
    d, n = private_key
    c = int.from_bytes(ciphertext, byteorder='big')
    m = pow(c, d, n)
    return m.to_bytes((n.bit_length() + 7) // 8, byteorder='big')

decrypted_message = rsa_decrypt(encrypted_message, private_key)
print("Decrypted Message:", decrypted_message)

# OAEP Decoding
decoded_message = oaep_decode(decrypted_message, 1024)
print("Decoded Message:", decoded_message)

# New Methods for Measuring Performance and Memory Usage
def measure_encryption_speed(message, public_key):
    start_time = time.time()
    encrypted_message = rsa_encrypt(message, public_key)
    end_time = time.time()
    return end_time - start_time, len(encrypted_message)

def measure_decryption_speed(ciphertext, private_key):
    start_time = time.time()
    decrypted_message = rsa_decrypt(ciphertext, private_key)
    end_time = time.time()
    return end_time - start_time, len(decrypted_message)

def measure_memory_usage():
    process = psutil.Process()
    memory_usage = process.memory_info().rss
    return memory_usage

# Measure encryption and decryption speeds
encryption_time, encrypted_length = measure_encryption_speed(encoded_message, public_key)
decryption_time, decrypted_length = measure_decryption_speed(encrypted_message, private_key)
memory_usage = measure_memory_usage()

print(f"Encryption Time: {encryption_time} seconds")
print(f"Decryption Time: {decryption_time} seconds")
print(f"Memory Usage: {memory_usage} bytes")

Public Key: (65537, 94245415515305315585778136442432312830770961726408695666906093444754604290288989509219696211096211640768155178368302254811219786607082718991758240217538409132085037745131824697968159259085449264186078098076255391853177282716584880931385128266868341023940202383560906612732376569136628304535679153823238178853)
Private Key: (40709728364477748125757866007733285684823766048383413425003350768688803162393625036490843035841168429108834779505138601575458457650791227733000961629587778763674839154691991113750431566227049240786856687192597458746268016536424953317291995696285034364966338140851375898024174295899032801515134749763812687613, 94245415515305315585778136442432312830770961726408695666906093444754604290288989509219696211096211640768155178368302254811219786607082718991758240217538409132085037745131824697968159259085449264186078098076255391853177282716584880931385128266868341023940202383560906612732376569136628304535679153823238178853)
Original Message: b'Hello, RSA with 